In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_scatter import scatter

from utils.dataset import *
from utils.clustering import get_cluster_embeddings

import os
import numpy as np
import pandas as pd
from collections import defaultdict

from einops import rearrange, reduce
from tqdm.auto import tqdm
from icecream import ic
from IPython.display import display

In [3]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [4]:
seed_everything(42)

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

module_activation_dict = {
    # conv blocks
    'audio_encoder.base.conv_block1': nn.Identity(),    # 0
    'audio_encoder.base.conv_block2': nn.Identity(),    # 1
    'audio_encoder.base.conv_block3': nn.Identity(),    # 2
    'audio_encoder.base.conv_block4': nn.Identity(),    # 3
    'audio_encoder.base.conv_block5': nn.Identity(),    # 4
    'audio_encoder.base.conv_block6': nn.Identity(),    # 5
    'audio_encoder.base.fc1': F.relu,                   # 6
    # 'audio_encoder.projection.linear1': F.gelu,         # 7
    # 'audio_encoder.projection.linear2': nn.Identity(),  # 8
}

module_list = list(module_activation_dict.keys())

In [5]:
cols = [
    # Cluster identity
    ('id', 'layer_name'),
    ('id', 'layer_idx'),
    ('id', 'partition_idx'),
    ('id', 'cluster_idx'),

    # Descriptive information
    ('desc', 'neuron_count'),
    ('desc', 'entropy'),
    ('desc', 'primary_class'),
    ('desc', 'secondary_class'),

    # Intervention results
    ('acc_drop', 'selective', 'primary_class'),
    ('acc_drop', 'selective', 'secondary_class'),
    ('acc_drop', 'selective', 'others'),

    ('acc_drop', 'random', 'primary_class'),
    ('acc_drop', 'random', 'secondary_class'),
    ('acc_drop', 'random', 'others'),
]

In [6]:
layer_name = module_list[5]
layer_name

'audio_encoder.base.conv_block6'

In [6]:
dataset = ESC50Dataset()

Using downloaded and verified file: /scratch/pratyaksh.g/esc50/ESC-50-master.zip


2000it [00:00, 11918.51it/s]

Loading audio files


In [7]:
class_indices = torch.LongTensor([dataset.class_to_idx[label] for label in dataset.class_label])

In [ ]:
 class_indices.device

In [130]:
data_dict = {
    'id': {
        'layer_name': [],
        'layer_idx': [],
        'partition_idx': [],
        'cluster_idx': [],
    },
    'desc': {
        'neuron_count': [],
        'entropy': [],
        'primary_class': [],
        'secondary_class': [],
    },
    'acc_drop': {
        'selective': {
            'primary_class': [],
            'secondary_class': [],
            'others': [],
        },
        'random': {
            'primary_class': [],
            'secondary_class': [],
            'others': [],
        },
    },
}

for layer_idx, layer_name in enumerate(tqdm(module_list)):
    # Get the df to iterate through clusters
    df = pd.read_csv(f'/scratch/pratyaksh.g/esc50/cluster-stats/{layer_name}.csv')
    threshold = 5.26
    df = df[df['entropy'] < threshold]
    df.sort_values('neuron_count', ascending=False, inplace=True)
    df = df.head(100)

    root = '/scratch/pratyaksh.g/'
    activations = torch.load(root + f'{dataset.path_name}/activations/{layer_name}.pt')
    clusters = torch.load(root + f'{dataset.path_name}/clusters/{layer_name}.pt')
    n_clusters = torch.load(root + f'{dataset.path_name}/clusters/{layer_name}_n.pt')
    cluster_embeddings = get_cluster_embeddings(activations, clusters, n_clusters)

    for idx, record in tqdm(df.iterrows(), total=len(df)):
        partition_idx, cluster_idx = record[['partition_idx', 'cluster_idx']]
        neuron_count = (clusters[:, partition_idx] == cluster_idx).sum().item()

        # Get the cluster's classwise embedding
        clust_embed = cluster_embeddings[partition_idx][cluster_idx]
        classwise_embed = scatter(clust_embed, class_indices, dim=0, reduce='sum')
        probs = F.normalize(classwise_embed, dim=0, p=1, eps=1e-8)
        entropy = - torch.sum(probs * torch.log2(probs + 1e-8)).item()

        # Get the primary and secondary class for the cluster
        primary_class_idx, secondary_class_idx = torch.argsort(classwise_embed, descending=True)[:2]
        primary_class_idx = primary_class_idx.item()
        primary_class = dataset.classes[primary_class_idx]
        secondary_class_idx = secondary_class_idx.item()
        secondary_class = dataset.classes[secondary_class_idx]
        other_labels = [label for label in dataset.classes if label not in [primary_class, secondary_class]]

        # Retrieve the intervention data
        cluster_path = lambda trt, inv: root + f'{dataset.path_name}/intervened-performance/{layer_name}/' + \
            f'partition-{partition_idx}-cluster-{cluster_idx}-{trt}-invert_mask={inv}.csv'

        normal = pd.read_csv(root + f'{dataset.path_name}/intervened-performance/normal-performance.csv')
        selective = pd.read_csv(cluster_path('intervened', 'false'))
        random = pd.read_csv(cluster_path('random', 'false'))

        # Compute the differences in classwise accuracies for the primary class,
        # the secondary class, as well as others, for both selective and random treatments.
        selective_diff = normal.groupby('label').mean()['accuracy'] - selective.groupby('label').mean()['accuracy']
        selective_primary_diff = selective_diff[primary_class]
        selective_secondary_diff = selective_diff[secondary_class]
        selective_others_diff = np.mean([selective_diff[label] for label in other_labels])

        random_diff = normal.groupby('label').mean()['accuracy'] - random.groupby('label').mean()['accuracy']
        random_primary_diff = random_diff[primary_class]
        random_secondary_diff = random_diff[secondary_class]
        random_others_diff = np.mean([random_diff[label] for label in other_labels])

        # Add everything appropriately to the dictionary
        data_dict['id']['layer_name'].append(layer_name)
        data_dict['id']['layer_idx'].append(layer_idx)
        data_dict['id']['partition_idx'].append(partition_idx)
        data_dict['id']['cluster_idx'].append(cluster_idx)

        data_dict['desc']['neuron_count'].append(neuron_count)
        data_dict['desc']['entropy'].append(entropy)
        data_dict['desc']['primary_class'].append(primary_class)
        data_dict['desc']['secondary_class'].append(secondary_class)

        data_dict['acc_drop']['selective']['primary_class'].append(selective_primary_diff)
        data_dict['acc_drop']['selective']['secondary_class'].append(selective_secondary_diff)
        data_dict['acc_drop']['selective']['others'].append(selective_others_diff)

        data_dict['acc_drop']['random']['primary_class'].append(random_primary_diff)
        data_dict['acc_drop']['random']['secondary_class'].append(random_secondary_diff)
        data_dict['acc_drop']['random']['others'].append(random_others_diff)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [10]:
# Step 1: Flatten the dictionary structure with consistent level depth
flattened_dict = {}
for outer_key, inner_dict in data_dict.items():
    if isinstance(inner_dict, dict):
        for inner_key, value in inner_dict.items():
            if isinstance(value, dict):
                for inner_inner_key, final_value in value.items():
                    flattened_dict[(outer_key, inner_key, inner_inner_key)] = final_value
            else:
                flattened_dict[(outer_key, inner_key, '')] = value
    else:
        flattened_dict[(outer_key, '', '')] = inner_dict

# Step 2: Create the MultiIndex
index = pd.MultiIndex.from_tuples(flattened_dict.keys(), names=['Level1', 'Level2', 'Level3'])

# Step 3: Create the DataFrame
data_frame = pd.DataFrame.from_dict(flattened_dict, orient='index').transpose()
data_frame.columns = index

# Display the DataFrame
data_frame

Level1                              id                                      \
Level2                      layer_name layer_idx partition_idx cluster_idx   
Level3                                                                       
0       audio_encoder.base.conv_block1         0             4         141   
1       audio_encoder.base.conv_block1         0             4         222   
2       audio_encoder.base.conv_block1         0             4         173   
3       audio_encoder.base.conv_block1         0             4         200   
4       audio_encoder.base.conv_block1         0             4         213   
..                                 ...       ...           ...         ...   
498             audio_encoder.base.fc1         6             0          95   
499             audio_encoder.base.fc1         6             0         170   
500             audio_encoder.base.fc1         6             0          46   
501             audio_encoder.base.fc1         6             0          99   
502             audio_encoder.base.fc1         6             0          63   

Level1         desc                                                acc_drop  \
Level2 neuron_count   entropy   primary_class secondary_class     selective   
Level3                                                        primary_class   
0              5034  5.242239  glass breaking         rooster           0.0   
1               640  5.243355         rooster  glass breaking           0.0   
2               362  5.195439     clock alarm             dog           0.0   
3               345  5.206063     clock alarm         rooster           0.0   
4               345  5.253973    church bells     crying baby           0.0   
..              ...       ...             ...             ...           ...   
498               6   3.32837            frog    toilet flush           0.0   
499               6  5.016367           train       sea waves           0.0   
500               6  3.895489      clock tick     mouse click        0.0175   
501               6  4.951536   pouring water          engine          0.02   
502               6  2.124309     mouse click       footsteps          -0.0   

Level1                                                                    
Level2                                  random                            
Level3 secondary_class    others primary_class secondary_class    others  
0                  0.0  0.005052           0.0             0.0 -0.002344  
1                  0.0 -0.000573           0.0             0.0 -0.000312  
2                  0.0 -0.000208           0.0             0.0 -0.000052  
3                  0.0  0.000156           0.0             0.0  0.000104  
4                  0.0 -0.000625           0.0             0.0  0.000104  
..                 ...       ...           ...             ...       ...  
498                0.0 -0.000625           0.0             0.0 -0.000156  
499                0.0 -0.000625           0.0             0.0 -0.000156  
500             0.0225 -0.000625          -0.0            -0.0 -0.000156  
501             0.0325 -0.000052           0.0             0.0 -0.000156  
502                0.0       0.0          -0.0         -0.0075       0.0  

[503 rows x 14 columns]

In [11]:
data_frame.to_csv('/scratch/pratyaksh.g/paper-plots/clusterwise-intervention-data.csv')

In [12]:
data_frame.sort_values(('acc_drop', 'selective', 'primary_class'), ascending=False, inplace=True)

In [13]:
data_frame.to_html('/scratch/pratyaksh.g/paper-plots/clusterwise-intervention-data.html')

,Level1,id,id.1,id.2,id.3,desc,desc.1,desc.2,desc.3,acc_drop,acc_drop.1,acc_drop.2,acc_drop.3,acc_drop.4,acc_drop.5
0,Level2,layer_name,layer_idx,partition_idx,cluster_idx,neuron_count,entropy,primary_class,secondary_class,selective,selective,selective,random,random,random
1,Level3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,primary_class,secondary_class,others,primary_class,secondary_class,others
2,0,audio_encoder.base.conv_block1,0,4,141,5034,5.242238998413086,glass breaking,rooster,0.0,0.0,0.00505208333333334,0.0,0.0,-0.0023437499999999895
3,1,audio_encoder.base.conv_block1,0,4,222,640,5.243354797363281,rooster,glass breaking,0.0,0.0,-0.0005729166666666603,0.0,0.0,-0.0003124999999999968
4,2,audio_encoder.base.conv_block1,0,4,173,362,5.195438861846924,clock alarm,dog,1.1102230246251565e-16,0.0,-0.00020833333333332543,1.1102230246251565e-16,0.0,-5.208333333333107e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,498,audio_encoder.base.fc1,6,0,95,6,3.3283700942993164,frog,toilet flush,1.1102230246251565e-16,0.0,-0.0006249999999999959,1.1102230246251565e-16,0.0,-0.0001562499999999961
501,499,audio_encoder.base.fc1,6,0,170,6,5.016366958618164,train,sea waves,1.1102230246251565e-16,0.0,-0.0006249999999999919,1.1102230246251565e-16,0.0,-0.0001562499999999961
502,500,audio_encoder.base.fc1,6,0,46,6,3.895489454269409,clock tick,mouse click,0.01749999999999996,0.022499999999999964,-0.0006249999999999936,-1.1102230246251565e-16,-1.1102230246251565e-16,-0.00015624999999998916
503,501,audio_encoder.base.fc1,6,0,99,6,4.951536178588867,pouring water,engine,0.02000000000000013,0.032500000000000084,-5.208333333332702e-05,1.1102230246251565e-16,1.1102230246251565e-16,-0.0001562499999999984


In [23]:
clusters_to_plot = [
    (6, 1, 7),
    (5, 4, 0),
    (5, 4, 28),
    (5, 4, 9),
]

In [41]:
import operator, functools

predicates = [(data_frame['id']['layer_idx'] == lidx) & (data_frame['id']['partition_idx'] == pidx) & (data_frame['id']['cluster_idx'] == cidx)
for lidx, pidx, cidx in clusters_to_plot]

data_frame[ predicates[0] | predicates[1] | predicates[2] | predicates[3] ]

Level1                              id                                      \
Level2                      layer_name layer_idx partition_idx cluster_idx   
Level3                                                                       
419             audio_encoder.base.fc1         6             1           7   
316     audio_encoder.base.conv_block6         5             4           0   
303     audio_encoder.base.conv_block6         5             4           9   
367     audio_encoder.base.conv_block6         5             4          28   

Level1         desc                                              acc_drop  \
Level2 neuron_count   entropy primary_class secondary_class     selective   
Level3                                                      primary_class   
419              20  4.581177           cat             dog        0.4425   
316            2943  4.926579      clapping    toilet flush          0.42   
303            9166  5.060297      hand saw  brushing teeth        0.3175   
367             925  4.945932     footsteps  crackling fire          0.14   

Level1                                                                    
Level2                                  random                            
Level3 secondary_class    others primary_class secondary_class    others  
419             0.0225  0.003802           0.0             0.0 -0.000781  
316                0.0  0.006406          0.02             0.0   0.01401  
303             0.1725  0.041146        0.0125           0.055  0.061146  
367                0.0  0.000521       -0.0075          0.0075  0.003385

In [125]:
print(data_frame[ predicates[0] | predicates[1] | predicates[2] | predicates[3] ].reset_index().to_latex(float_format="%.2f"))

NameError: name 'data_frame' is not defined

In [46]:
print(data_frame.sample(n=4, random_state=42).to_latex())

\begin{tabular}{lllllllllllllll}
\toprule
Level1 & \multicolumn{4}{l}{id} & \multicolumn{4}{l}{desc} & \multicolumn{6}{l}{acc\_drop} \\
Level2 &                      layer\_name & layer\_idx & partition\_idx & cluster\_idx & neuron\_count &   entropy &     primary\_class & secondary\_class & \multicolumn{3}{l}{selective} & \multicolumn{3}{l}{random} \\
Level3 & primary\_class & secondary\_class &    others & primary\_class & secondary\_class &    others \\
\midrule
127 &  audio\_encoder.base.conv\_block3 &         2 &             3 &          39 &           36 &  4.888887 &             siren &            crow &           0.0 &          0.0025 &  -0.00026 &           0.0 &          0.0025 & -0.000625 \\
369 &  audio\_encoder.base.conv\_block6 &         5 &             3 &          47 &          912 &  3.982392 &               hen &         rooster &          0.05 &             0.0 & -0.000208 &         0.005 &             0.0 &  0.003125 \\
497 &          audio\_encoder.base.fc1 &      

In [ ]:
###

In [9]:
df = pd.read_csv(f'/scratch/pratyaksh.g/esc50/cluster-stats/{layer_name}.csv')
threshold = 5.26
df = df[df['entropy'] < threshold]
df = df.head(100)
df.sort_values('neuron_count', ascending=False, inplace=True)

In [10]:
root = '/scratch/pratyaksh.g/'
activations = torch.load(root + f'{dataset.path_name}/activations/{layer_name}.pt')
clusters = torch.load(root + f'{dataset.path_name}/clusters/{layer_name}.pt')
n_clusters = torch.load(root + f'{dataset.path_name}/clusters/{layer_name}_n.pt')
cluster_embeddings = get_cluster_embeddings(activations, clusters, n_clusters)

In [11]:
idx = 2
partition_idx, cluster_idx = df.loc[idx][['partition_idx', 'cluster_idx']]

In [12]:
partition_idx, cluster_idx

(5, 2)

In [13]:
clust_embed = cluster_embeddings[partition_idx][cluster_idx]
classwise_embed = scatter(clust_embed, class_indices, dim=0, reduce='sum')

In [14]:
primary_class_idx, secondary_class_idx = torch.argsort(classwise_embed, descending=True)[:2]

In [15]:
primary_class_idx = primary_class_idx.item()
secondary_class_idx = secondary_class_idx.item()

In [41]:
primary_class = dataset.classes[primary_class_idx]
secondary_class = dataset.classes[secondary_class_idx]

In [25]:
cluster_path = lambda trt, inv: root + f'{dataset.path_name}/intervened-performance/{layer_name}/partition-{partition_idx}-cluster-{cluster_idx}-{trt}-invert_mask={inv}.csv'

normal = pd.read_csv(root + f'{dataset.path_name}/intervened-performance/normal-performance.csv')
selective = pd.read_csv(cluster_path('intervened', 'false'))
random = pd.read_csv(cluster_path('random', 'false'))

In [29]:
selective_diff = normal.groupby('label').mean()['accuracy'] - selective.groupby('label').mean()['accuracy']

In [59]:
selective_diff[dataset.classes[primary_class_idx]], selective_diff[dataset.classes[secondary_class_idx]]

(0.3825, 0.255)

In [57]:
np.mean([selective_diff[label] for label in dataset.classes if label != primary_class and label != secondary_class])

0.0061979166666666875

In [60]:
random_diff = normal.groupby('label').mean()['accuracy'] - random.groupby('label').mean()['accuracy']

In [61]:
random_diff[dataset.classes[primary_class_idx]], random_diff[dataset.classes[secondary_class_idx]]

(0.04500000000000004, 0.04249999999999987)

In [62]:
np.mean([random_diff[label] for label in dataset.classes if label != primary_class and label != secondary_class])

0.03239583333333335